# Установка библиотек и функций

In [12]:
#@title Установка библиотек
!pip  install  tiktoken cohere langchain==0.0.347 openai==0.28.0 faiss-cpu
!pip install git+https://github.com/burnash/gspread.git  # Библиотека для работы с Google Таблицами через API Google
!pip install google
!pip install --upgrade google-api-python-client
!pip install openpyxl


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     ------ ------------------------------ 41.0/250.0 kB 960.0 kB/s eta 0:00:01
     --------------------- ---------------- 143.4/250.0 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
#@title Импорт библиотек
import openai
import tiktoken

from langchain.llms import OpenAI

from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

import os
import json
import requests
from dotenv import load_dotenv
# from google.colab import userdata

import IPython

from google.oauth2.service_account import Credentials
# import gspread
from openpyxl.utils import get_column_letter
import re
import pprint

In [14]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.001, 0.002, 'GPT 3.5']  # 16K context window, [1], [2] - price per 1k tokens
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002, 'GPT 3.5']  # 4,096 tokens, Shutdown date 2024-06-13
MODEL_GPT_3_5_TURBO_INSTRUCT = ['gpt-3.5-turbo-instruct', 0.0015, 0.002, 'GPT 3.5']  # 4,096 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06,'GPT 4']  # 8,192 tokens
MODEL_GPT_4_TURBO = ['gpt-4-1106-preview', 0.01, 0.03,'GPT 4']  # 128k tokens	content
# gpt-4-32k-0613    Up to Apr 2023 The latest GPT-4 model with improved instruction following, JSON mode, reproducible outputs, parallel function calling, and more. Returns a maximum of 4,096 output tokens.
# gpt-3.5-turbo-1106  Updated GPT 3.5 TurboNew The latest GPT-3.5 Turbo model with improved instruction following, JSON mode, reproducible outputs, parallel function calling, and more. Returns a maximum of 4,096 output tokens. 16,385 tokens
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K # выбери нужную модель

In [31]:
# добавить в Секрет Колаба (ключ слева на панели) переменные окружения OPENAI_API_KEY
# API configuration
# получим переменные окружения из .env
dotenv_path = 'E:/GPT/Lesson_11/.env'
load_dotenv(dotenv_path)

# API-key
openai.api_key = os.environ.get("OPENAI_API_KEY")  # Вариант для VSCode
# for LangChain
# os.environ['OPENAI_API_KEY'] = openai_key  # Вариант для Колаб

In [34]:
#@title  Функция для загрузки файла по ссылке из гугл драйв
def download_file_from_google_drive(remote_filepath, local_filepath):
    # Извлекаем ID документа из url
    match_ = re.search('/file/d/([a-zA-Z0-9-_]+)', remote_filepath)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Загружаем файл
    response = requests.get(f'https://drive.google.com/uc?export=download&id={doc_id}')
    # Сохраняем файла на диск
    with open(local_filepath, "wb") as f:
        f.write(response.content)

In [36]:
#@title  Функции чтения и записи диапазонов строк из таблиц Google Sheets

def read_range_from_spreadsheet(sheet_url, sheet_name, column_name, start_row, end_row):
    scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file('E:/GPT/WB/wb_sheet_api.json',scopes=scopes)
    # Авторизуемся
    client = gspread.authorize(credentials)
    sheet = client.open_by_url(sheet_url).worksheet(sheet_name)

    # Получаем заголовки колонок
    headers = sheet.row_values(1)
    column_index = headers.index(column_name) + 1  # Получаем индекс колонки по ее имени

    # Читаем диапазон строк из указанной колонки
    range_values = sheet.get(f'{get_column_letter(column_index)}{start_row}:' \
                             f'{get_column_letter(column_index)}{end_row}')

    return [row[0] for row in range_values] if range_values else None

def write_range_to_spreadsheet(sheet_url, sheet_name, column_name, start_row, end_row, values):
    scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file('E:/GPT/WB/wb_sheet_api.json',scopes=scopes)
    # Авторизуемся
    client = gspread.authorize(credentials)
    sheet = client.open_by_url(sheet_url).worksheet(sheet_name)

    sheet = client.open_by_url(sheet_url).worksheet(sheet_name)

    # Получаем заголовки колонок
    headers = sheet.row_values(1)
    column_index = headers.index(column_name) + 1  # Получаем индекс колонки по ее имени

    # Подготавливаем список списков для обновления диапазона ячеек
    update_list = []
    # Подготавливаем список списков для обновления диапазона ячеек
    update_list = [[value] for value in values[:end_row - start_row + 1]]  # Ограничиваем количество значений до end_row

    # Обновляем диапазон ячеек указанной колонки
    result = sheet.update(f'{get_column_letter(column_index)}{start_row}', update_list)
    return result

In [37]:
#@title  Функция для подсчета количества токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

In [38]:
#@title  Функция загрузки базы знаний из файла и деления ее на чанки
def load_documents(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Header1"),
        ("##", "Header2"),
        ("###", "Header3"),
        ("####", "Header4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    documents = markdown_splitter.split_text(text)

    return documents

In [39]:
#@title  Функция для загрузки API-ключа сервисного аккаунта Google
def Load_Google_API_key (url_key, filepath_key) :
    # Загрузка API-ключа сервисного аккаунта Google
    url_api_key     = url_key
    local_filepath  = filepath_key
    # Загружаем файл по ссылке и сохраняем в папке local_filepath
    download_file_from_google_drive(url_api_key, local_filepath)

In [40]:
# Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
#@title  Функция создания эмбеддингов и их сохранения в векторную базу
def create_vector_base(documents: Document):
    vectordateBase = FAISS.from_documents(documents, OpenAIEmbeddings())   # ada v2	$0.0001 / 1K tokens

    count_token = num_tokens_from_string(' '.join([x.page_content for x in documents]), "cl100k_base")
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0001 * (count_token / 1000), ' $')

    return vectordateBase

In [41]:
#@title Загрузка базы знаний
#!git clone -b knowledge https://github.com/terrainternship/GPT_Wildberries_g.git
#!cp -r /content/GPT_Wildberries_g/knowledge_base/output/knowledge_base.md /content/knowledge_base.md
knowledge_base = 'E:/GPT/WB/GPT_Wildberries_g/knowledge_base/output/knowledge_base.md'
# !rm -r /content/GPT_Wildberries_g

In [42]:
documents = load_documents(knowledge_base)

In [44]:
#@title System prompt
prompt = '''
Вы являетесь нейро-консультантом техподдержки, специализирующимся
на обслуживании клиентов ООО "Вайлдберриз". Ваша задача - предоставлять точные
и полезные ответы на вопросы клиентов, связанные с техническими аспектами работы
сервиса, на основе обширной базы внутренних документов, инструкций и правил
компании. Вы обладаете глубокими знаниями о продуктах и услугах компании,
а также о соответствующих процедурах и правилах. Ваши ответы должны быть ясными,
конкретными и полезными, а также соответствовать официальной политике и
стандартам компании.
'''

In [17]:
#client = OpenAI(api_key=openai_key)

In [45]:
# Создание векторной базы
knowledge_base = create_vector_base(documents)

Количество токенов в документе : 52041
ЦЕНА запроса: 0.0052041  $


In [46]:
def answer_index(topic, knowledge_base=knowledge_base,temp=0.1, top_similar_documents=3):

    docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)

    # создадим список найденных документов
    responses = []
    for i, (doc, score) in enumerate(docs):
        if score < 2: # праметр Л2 для чанков. 0..1
            content = doc.page_content

            response = f' ___ score = {score} ___ Metadata документа = {doc.metadata}\n___Отрывок документа №{i + 1}___\n{content}\n'
            count_token = num_tokens_from_string(response, "cl100k_base")
            # раскомментируй строки ниже для вывода на экран найденных чанков
            # print(f'\n_____Отрывок документа №{i + 1}______\n')
            # print(f'___ score = {score} --- Metadata документа --- {doc.metadata}')
            # print(f'\n{content}\n')

            responses.append(f'__tokens = {count_token}__{response}')

    # создадим формат запроса
    messages = [
        {"role": "system", "content": prompt},
        # реализация памяти:
        # {"role": "user", "content": history_question},
        # {"role": "assistant", "content": history_answear},
        {"role": "user", "content": f"Документы с информацией для ответа клиенту: {responses}\n\nВопрос клиента: \n{topic}"}
    ]

    # completion = client.chat.completions.create(
    # model=SELECT_MODEL_GPT[0],
    # messages=messages,
    # temperature=temp,
    # max_tokens=15000
    # )
    completion = openai.ChatCompletion.create(
        model=SELECT_MODEL_GPT[0],
        messages=messages,
        temperature=temp,
        max_tokens=15000
    )

    # print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    # print('ЦЕНА запроса с ответом :', (SELECT_MODEL_GPT[1])*(completion["usage"]["prompt_tokens"]/1000) + (SELECT_MODEL_GPT[2])*(completion["usage"]["completion_tokens"]/1000), ' $')
    # print('===========================================: \n')

    answer = completion.choices[0].message.content

    # return  IPython.display.Markdown(answer)
    return answer, responses

In [47]:
#@title Функция для запуска режима диалога в формате "Вопрос-Ответ"
def run_dialog():
    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer,chank = answer_index(user_question)
        print('\nМенеджер: ', answer + '\n\n')

    return

In [48]:
#@title Загрузка API-ключа сервисного аккаунта Google в сессионное хранилище Colab
url_api_key     = 'https://drive.google.com/file/d/1MJCzpsu9i4eKRVBhHB-_ndfGTPFZTLzD/view?usp=drive_link'
local_filepath  = "E:/GPT/WB/wb_sheet_api.json"

Load_Google_API_key(url_api_key,local_filepath)

In [50]:
#@title Функция последовательного чтения вопросов из Google Sheets файла и записи ответов и чанков к ответам
def run_dialog_from_file(sheet_url,row_start,row_end):
    # Читаем вопросы из диапазона строк
    questions = read_range_from_spreadsheet(sheet_url, 'Вопросы 14.12', 'Вопрос',row_start, row_end)
    answers = []
    chunks = []
    models = []
    # Получаем ответы на вопросы и создаем список ответов
    for question in questions :
          print('Вопрос: ',question+'\n')
          # На случай различных ошибок, чтобы не прерывать цикл и не терять ответы на предыдущие вопросы
          try:
              answer,chank = answer_index(question)
          except Exception as e:
              answer = f'\n\nПопытка ответа на вопрос {question} не удалась из-за ошибки: {str(e)}'
              # Повторяем поиск чанков, чтобы их записать в таблицу
              docs = knowledge_base.similarity_search_with_score(question, k=3)
              # создадим список найденных чанков
              chank = []
              for i, (doc, score) in enumerate(docs):
                  if score < 2: # праметр Л2 для чанков. 0..1
                      content = doc.page_content
                      ch = f' ___ score = {score} ___ Metadata документа = {doc.metadata}\n___Отрывок документа №{i + 1}___\n{content}\n'
                      count_token = num_tokens_from_string(ch, "cl100k_base")
                      chank.append(f'__tokens = {count_token}__{ch}')

          print('Ответ: ',answer+'\n\n')
          answers.append(answer)
          models.append(SELECT_MODEL_GPT[3]) # Имя модели
          # Собираем все чанки в один текст
          ch_text =''
          for cha in chank :
            ch_text += f'\n____{cha}______\n'
            chunks.append(ch_text)

    # Записываем ответы в соответствующий диапазон ячеек
    write_range_to_spreadsheet(sheet_url, 'Вопросы 14.12', 'Тестовый ответ GPT', row_start, row_end, answers)
    # Записываем версию GPT
    write_range_to_spreadsheet(sheet_url, 'Вопросы 14.12', 'Версия GPT ', row_start, row_end, models)
    # Записываем чанки в соответствующий диапазон ячеек
    # print(chunks)
    write_range_to_spreadsheet(sheet_url, 'Вопросы 14.12', 'Чанки', row_start, row_end, chunks)

In [ ]:
#@title Запуск режима диалога с выводом на экран
run_dialog()


Клиент: Как мне получить цифровой товар?

Менеджер:  Для получения цифрового товара на площадке Digital.Wildberries.ru, вам необходимо совершить оплату товара на сайте. После этого вам автоматически откроется доступ к приобретенному товару/услуге. Вы также получите письмо на указанную вами почту с ссылкой на приобретенный товар/услугу. Купленный контент будет доступен сразу после оплаты на странице товара или в разделе "Мои покупки".



Клиент: stop


In [51]:
#@title Запуск режима отработки вопросов по файлу
# Укажите сюда ссылку на Ваш файл вопросов и ответов. Не забудьте открыть к нему общий доступ по ссылке
url = 'https://docs.google.com/spreadsheets/d/1yUBr0u9YaYOFxW8FEs1ptm6rCBeSdOmBZkbWZq1y1LI/edit?usp=sharing'
# Укажите строку с началом и концом диапазона вопросов. При повторном вызове ответы будут переписываться !
row_start  = 6
row_end = 6
#################################Запуск отработки#########################################
run_dialog_from_file(url,row_start,row_end)

FileNotFoundError: [Errno 2] No such file or directory: '/content/wb_sheet_api.json'